<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/Alesya-Moroz-folder/03(5)_profpatolog_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек

In [99]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import tensorflow as tf


from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential, Model # Полносвязная модель
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input # Слои для сети
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами и преобразования их в последовательности
from tensorflow.keras.preprocessing.sequence import pad_sequences # Метод для работы с последовательностями

from tensorflow.keras.layers import TextVectorization

import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from pandas.core.common import flatten

from sklearn.metrics import f1_score

In [2]:
print(tf.__version__)
print(tf.keras.__version__)

2.12.0
2.12.0


# Загрузка датасета

In [3]:
from google.colab import drive # Подключаем диск
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/MyDrive/NU_files/стажировка/profpatolog_modifiedDataset_v1.xlsx', index_col=0)

In [5]:
df.head()

,КлиентПолПредставление,Возраст,ДиспансернаяГруппаПредставление,Профессия,ВредныеФакторы,ЗаключениеМК,ЛабораторныеИсследования1_ОАК_ЗначениеПредставление,ЛабораторныеИсследования1_Биохимия_ЗначениеПредставление,ЛабораторныеИсследования1_ОАМ_ЗначениеПредставление,ЛабораторныеИсследования1_МазокИзЗеваИНоса_ЗначениеПредставление,...,Психиатрия1_МКБ102,Психиатрия2_МКБ101,Психиатрия2_МКБ102,Рентгенология1_МКБ102,Рентгенология2_МКБ102,Психиатрия3_МКБ101,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_МКБ102,Оториноларингология3_МКБ101
0,Женский,45,I,Ведущий специалист по организации обуения,11.4,Годен,получен,получен,получен,не получен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Мужской,34,IIIа,Ведущий специалист,4.2.5,Годен,получен,получен,получен,не получен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Мужской,36,II,Главный менеджер,4.2.5,ГоденСКоррекциейЗрения,получен,получен,получен,не получен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Мужской,33,I,Главный специалист,11.1,Годен,получен,получен,получен,получен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Мужской,37,II,Старший специалист,11.1,ГоденСКоррекциейЗрения,получен,получен,получен,не получен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info()
# df.info(verbose=True, buf=None, max_cols=4, memory_usage=None, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 0 to 1570
Columns: 1626 entries, КлиентПолПредставление to Оториноларингология3_МКБ101
dtypes: float64(87), int64(1), object(1538)
memory usage: 19.5+ MB


In [7]:
#@title Столбцы с общей информацией
print(list(df.columns[:6]))

['КлиентПолПредставление', 'Возраст', 'ДиспансернаяГруппаПредставление', 'Профессия', 'ВредныеФакторы', 'ЗаключениеМК']


In [8]:
# n = 0 # номер записи
# for i in range(len(df.values[n])):
#     if df.values[n][i] != np.NaN:
#         print(i, df.columns[i], " ", df.values[n][i])

In [9]:
#@title Определение направлений исследований (по столбцам)
columns = df.columns
col_specialization = []
for i, column in enumerate(columns):
    string = re.split('_|[0-9]', column)
    if len(string) > 1:
        if string[0] not in col_specialization: 
            col_specialization.append(string[0])    
            print(len(col_specialization), string[0])

1 ЛабораторныеИсследования
2 Рентгенология
3 ПсихиатрияНаркология
4 Офтальмология
5 Оториноларингология
6 ХирургНеИсп
7 Гинекология
8 ФункциональнаяДиагностика
9 Стоматология
10 Эндоскопия
11 Терапия
12 Дерматовенерология
13 Неврология
14 УЗИДиагностика
15 Хирургия
16 ЭКГНеИсп
17 ПроцедурнаяМедсестра
18 Психиатрия
19 ТерапевтНеИсп


## Формирование датасета с заключениями специалистов (МКБ)

In [10]:
keywords_to_save = 'МКБ'
df_mkb = df.filter(regex=keywords_to_save)
df_mkb.info()
columns_mkb = list(df_mkb.columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 0 to 1570
Data columns (total 91 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ПсихиатрияНаркология1_МКБ101       1208 non-null   object
 1   Офтальмология1_МКБ101              1005 non-null   object
 2   Оториноларингология1_МКБ101        1221 non-null   object
 3   ХирургНеИсп1_МКБ101                25 non-null     object
 4   Гинекология1_МКБ101                363 non-null    object
 5   Стоматология1_МКБ101               321 non-null    object
 6   Дерматовенерология1_МКБ101         754 non-null    object
 7   Рентгенология1_МКБ101              71 non-null     object
 8   Неврология1_МКБ101                 1355 non-null   object
 9   Терапия1_МКБ101                    1358 non-null   object
 10  Хирургия1_МКБ101                   785 non-null    object
 11  Офтальмология1_МКБ102              114 non-null    object
 12  Терапи

In [11]:
len(df_mkb.columns)

91

### Объединение МКБ столбцов

In [12]:
df['mkb_all'] = df[columns_mkb].astype(str).agg(','. join , axis= 1 ) 

In [13]:
p = df['mkb_all'][5]
type(p)
p

'nan,H52.2,Z00.0,nan,nan,Z00.0,nan,nan,Z00.0,Z00.0,Z00.0,H53.0,Z00.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan'

In [14]:
def mkb_all(arg):
    new = re.split(r',', arg)
    unique = list(set(new))
    unique.remove('nan')
    return unique

In [15]:
df['mkb_all_unique'] = df['mkb_all'].apply(mkb_all)

In [16]:
df['mkb_all_unique'][100]

['H52.1', 'Z00.0']

## Формирование датасета с заключениями специалистов (Профнепригодность)

In [17]:
# Профнепригодность
df_prof = df.filter(regex='Профнепригодность')
# df_prof.info()
df_prof

,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,Офтальмология1_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ПрофнепригодностьВременнаяПредставление1,ХирургНеИсп1_ПрофнепригодностьВременнаяПредставление1,Гинекология1_ПрофнепригодностьВременнаяПредставление1,Стоматология1_ПрофнепригодностьВременнаяПредставление1,Дерматовенерология1_ПрофнепригодностьВременнаяПредставление1,Рентгенология1_ПрофнепригодностьВременнаяПредставление1,Неврология1_ПрофнепригодностьВременнаяПредставление1,Терапия1_ПрофнепригодностьВременнаяПредставление1,...,Психиатрия1_ПрофнепригодностьВременнаяПредставление2,Психиатрия2_ПрофнепригодностьВременнаяПредставление1,Психиатрия2_ПрофнепригодностьВременнаяПредставление2,Рентгенология1_ПрофнепригодностьВременнаяПредставление2,Рентгенология2_ПрофнепригодностьВременнаяПредставление2,Психиатрия3_ПрофнепригодностьВременнаяПредставление1,Психиатрия3_ПрофнепригодностьВременнаяПредставление2,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_ПрофнепригодностьВременнаяПредставление2,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1
0,NaN,NaN,NaN,NaN,Годен,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Годен,Годен,NaN,NaN,NaN,NaN,Годен,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Годен,Годен,NaN,NaN,NaN,NaN,Годен,NaN,NaN,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Годен,Годен,Годен,NaN,Годен,NaN,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Годен,Годен,NaN,NaN,Годен,NaN,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,Годен,Годен,Годен,NaN,NaN,NaN,NaN,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1567,Годен,NaN,Годен,NaN,NaN,NaN,NaN,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568,Годен,Годен,Годен,NaN,NaN,NaN,NaN,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1569,Годен,Годен,Годен,NaN,NaN,NaN,NaN,NaN,Годен,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Формирование справочников и функций для парсинга данных

Для универсальности используем токенайзер, во избежание переписывания функций при изменении входных данных (например добавлении или удалении каких-либо параметров) 

In [18]:
#@title Справочник 'КлиентПолПредставление'

vocabulary_sex = list(df['КлиентПолПредставление'].astype(str).unique())
try: 
    vocabulary_sex.remove('nan')
except: 
    pass
print(vocabulary_sex)

['Женский', 'Мужской']


In [19]:
#@title Справочник 'ДиспансернаяГруппаПредставление'

vocabulary_dispensary = list(df['ДиспансернаяГруппаПредставление'].astype(str).unique())
try: 
    vocabulary_dispensary.remove('nan')
except: 
    pass
print(vocabulary_dispensary)

['I', 'IIIа', 'II', 'IIIб']


### Справочник 'ВредныеФакторы'
В данном ноутбуке справочник 'ВредныеФакторы' формируется на основе нормативной документации. Далее можно использовать справочник, который используется в клинике.

In [20]:
df_factors = pd.read_csv('/content/drive/MyDrive/NU_files/стажировка/Вредные факторы_справочник - Лист1 - Вредные факторы_справочник - Лист1.csv')
df_factors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       230 non-null    object
dtypes: object(1)
memory usage: 1.9+ KB


In [21]:
df_factors['1'][0]

'1.1. Азота неорганические соединения (в том числе азота оксиды О1, азота диоксид О).'

In [22]:
def factors_find(arg):
    # string = df_factors['1'][200]
    factors = re.split(r' ', arg)
    
    return factors[0][:][:-1]

In [23]:
df_factors['2'] = df_factors['1'].apply(factors_find)
df_factors['2'][0]

'1.1'

In [24]:
vocabulary_hfactor = df_factors['2'].to_list()
print(vocabulary_hfactor[:10], '\n')
print('Длина справочника ВредныеФакторы: ',len(vocabulary_hfactor))

['1.1', '1.2', '1.3', '1.4', '1.5', '1.5.1', '1.6', '1.7', '1.7.1', '1.7.2'] 

Длина справочника ВредныеФакторы:  230


### Справочник 'МКБ'
В данном ноутбуке справочник 'МКБ' формируется на основе нормативной документации. Далее можно использовать справочник, который используется в клинике.

In [25]:
df_mkb_catalog = pd.read_excel('/content/drive/MyDrive/NU_files/стажировка/spr_mkb10.xls')
df_mkb_catalog .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12941 entries, 0 to 12940
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   mkb_cod   12941 non-null  object
 1   mkb_name  12941 non-null  object
 2   class_id  12941 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 303.4+ KB


In [26]:
df_mkb_catalog['mkb_cod'][:5]

0      A00
1    A00.0
2    A00.1
3    A00.9
4      A01
Name: mkb_cod, dtype: object

In [27]:
vocabulary_mkb = df_mkb_catalog['mkb_cod'].to_list()
print(vocabulary_mkb[:10], '\n')
print('Длина справочника МКБ: ',len(vocabulary_mkb))

['A00', 'A00.0', 'A00.1', 'A00.9', 'A01', 'A01.0', 'A01.1', 'A01.2', 'A01.3', 'A01.4'] 

Длина справочника МКБ:  12941


### Справочник для векторизации

In [28]:
vocabulary = vocabulary_sex + vocabulary_dispensary + vocabulary_hfactor + vocabulary_mkb

print(vocabulary[:10])
print(vocabulary[230:240])
print('Длина общего справочника: ',len(vocabulary))


['Женский', 'Мужской', 'I', 'IIIа', 'II', 'IIIб', '1.1', '1.2', '1.3', '1.4']
['22', '23', '24', '25', '26', '27', 'A00', 'A00.0', 'A00.1', 'A00.9']
Длина общего справочника:  13177


## Tokenizer

In [62]:
max_tokens = len(vocabulary)+1
tokenizer = Tokenizer(num_words=max_tokens, 
                             filters='!"#$%&()*+-–—/…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', 
                             lower=False, 
                             split=',', 
                             oov_token='unknown', 
                             char_level=False)


tokenizer.fit_on_texts(vocabulary)

In [111]:
# print(tokenizer.word_index)

In [68]:
def tokenization_x(arg):
    # for i in arg:
    #     print(i)
    k = tokenizer.texts_to_sequences(arg)
    n = tokenizer.sequences_to_matrix(k)
    f = sum(n)
    return f

# Формирование выборок

##  xTrain

In [32]:
columns_xtrain = ['КлиентПолПредставление', 'ДиспансернаяГруппаПредставление', 'ВредныеФакторы'] + columns_mkb

In [33]:
print(len(columns_xtrain))
print(columns_xtrain[:5])

94
['КлиентПолПредставление', 'ДиспансернаяГруппаПредставление', 'ВредныеФакторы', 'ПсихиатрияНаркология1_МКБ101', 'Офтальмология1_МКБ101']


In [34]:
df['x_raw'] = df[columns_xtrain].astype(str).agg(','. join , axis= 1 ) 

In [35]:
p = df['x_raw'][5]
type(p)
print(p)
print(len(p))
row_length = len(p)

Мужской,II,11.1,nan,H52.2,Z00.0,nan,nan,Z00.0,nan,nan,Z00.0,Z00.0,Z00.0,H53.0,Z00.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
395


In [36]:
d = re.split(r',', p)

In [37]:
len(d)

94

In [38]:
def x_train_cleandata(arg):
    splitted = re.split(r',', arg)
    # unique = splitted

    
    unique = []
    for i in splitted:
        if i not in unique:
            unique.append(i)
    unique.remove('nan') # убираем из значений nan
    # while len(unique) < 30:
    #     unique.append('0')
    # unique = tuple(unique)
    # unique = np.asarray(unique)
    # unique = str(unique) # переводим значения в строки
    
    return unique

In [39]:
df['x_clean'] = df['x_raw'].apply(x_train_cleandata)

In [40]:
x_whole = df['x_clean']

In [72]:
arg = x_whole[3]
arg

['Мужской', 'I', '11.1', 'Z00.0']

In [67]:
k = tokenizer.texts_to_sequences(arg)
n = tokenizer.sequences_to_matrix(k)
f = sum(n)

In [71]:
tokenization_x(x_whole[3])

array([0., 0., 0., ..., 0., 0., 0.])

In [73]:
x_train = x_whole.map(tokenization_x)

print(type(x_train))
print(x_train.dtypes)

<class 'pandas.core.series.Series'>
object


In [74]:
x_train = x_train.apply(lambda x: pd.Series(flatten(x)))
x_train_np = x_train.to_numpy()

print(type(x_train_np))
print(x_train_np)
print(x_train_np.shape)

<class 'numpy.ndarray'>
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1571, 13178)


## yTrain

In [75]:
columns[:6]

Index(['КлиентПолПредставление', 'Возраст', 'ДиспансернаяГруппаПредставление',
       'Профессия', 'ВредныеФакторы', 'ЗаключениеМК'],
      dtype='object')

In [76]:
df['ЗаключениеМК'].unique()

array(['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах',
       'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден',
       'ОграниченноГоден'], dtype=object)

In [77]:
df['ЗаключениеМК'].value_counts()

Годен                               1434
ГоденСКоррекциейЗрения               123
ВременноНегоден                        9
НуждаетсяВДообследованииИЛечении       2
ОграниченноГоден                       2
ГоденБезРаботНаВысотах                 1
Name: ЗаключениеМК, dtype: int64

In [78]:
limited_eff = ['ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'ОграниченноГоден']

In [79]:
def y_train(arg):
    
    limited_eff = ['ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'ОграниченноГоден']
    
    if arg == 'Годен': y = 0
    
    elif arg in limited_eff: y = 1
    # elif arg == 'ГоденБезРаботНаВысотах': y = 1
    # elif arg == 'ОграниченноГоден': y = 1

    else: y = 2

    return utils.to_categorical(y, num_classes=3)

In [80]:
df['y_train'] = df['ЗаключениеМК'].apply(y_train)

In [81]:
print(df['y_train'][:5], '\n')
# print('Количество значений в каждой группе:')
# print(df['y_train'].value_counts())

0    [1.0, 0.0, 0.0]
1    [1.0, 0.0, 0.0]
2    [0.0, 1.0, 0.0]
3    [1.0, 0.0, 0.0]
4    [0.0, 1.0, 0.0]
Name: y_train, dtype: object 



In [82]:
y_whole = df['y_train']

In [84]:
y_train = y_whole.apply(lambda x: pd.Series(flatten(x)))
y_train_np = y_train.to_numpy()

print(type(y_train_np))
print(y_train_np)
print(y_train_np.shape)

<class 'numpy.ndarray'>
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
(1571, 3)


## Обучающая и тестовая выборка

In [85]:
# x_whole, y_whole = df['x_clean'], df['y_train']

x_train_dataset, x_test_dataset, y_train_dataset, y_test_dataset = train_test_split(
                                                                                    x_train_np, y_train_np,
                                                                                    train_size=0.8,
                                                                                    test_size=0.2,
                                                                                    random_state=42,
                                                                                    shuffle = False,      # перемешиваем выборку (нет)
                                                                                    stratify=None    # равномерно распределяем по категорииям (нет)
    )

print(x_whole[0], y_whole[0])
print()
print('x_whole.shape', x_whole.shape)
print('y_whole.shape', y_whole.shape)
print()
print('x_train.shape', x_train_dataset.shape, type(x_train_dataset[0]),type(x_train_dataset))
print('y_train.shape', y_train_dataset.shape, type(y_train_dataset[0]),type(y_train_dataset))
print()
print('x_test.shape:', x_test_dataset.shape)
print('y_test.shape:', y_test_dataset.shape)

['Женский', 'I', '11.4', 'Z00.0'] [1. 0. 0.]

x_whole.shape (1571,)
y_whole.shape (1571,)

x_train.shape (1256, 13178) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
y_train.shape (1256, 3) <class 'numpy.ndarray'> <class 'numpy.ndarray'>

x_test.shape: (315, 13178)
y_test.shape: (315, 3)


## Нейросеть (1)

In [91]:
model_1 = Sequential()
model_1.add(Dense(32, activation = 'relu', input_shape=(13178,)))
model_1.add(Dense(32, activation = 'relu'))
model_1.add(Dense(3, activation = 'softmax'))

model_1.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                421728    
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 422,883
Trainable params: 422,883
Non-trainable params: 0
_________________________________________________________________


### metrics=['accuracy']

In [94]:
model_1.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

In [95]:
history = model_1.fit(x_train_dataset, y_train_dataset, batch_size=32, epochs=30, validation_data=(x_test_dataset, y_test_dataset))

Epoch 1/30
40/40 [==============================] - 1s 14ms/step - loss: 0.8036 - accuracy: 0.9005 - val_loss: 0.3794 - val_accuracy: 0.9365
Epoch 2/30
40/40 [==============================] - 0s 10ms/step - loss: 0.3597 - accuracy: 0.9068 - val_loss: 0.2658 - val_accuracy: 0.9365
Epoch 3/30
40/40 [==============================] - 0s 10ms/step - loss: 0.2918 - accuracy: 0.9068 - val_loss: 0.2404 - val_accuracy: 0.9365
Epoch 4/30
40/40 [==============================] - 0s 9ms/step - loss: 0.2533 - accuracy: 0.9068 - val_loss: 0.2256 - val_accuracy: 0.9365
Epoch 5/30
40/40 [==============================] - 0s 8ms/step - loss: 0.2311 - accuracy: 0.9068 - val_loss: 0.2181 - val_accuracy: 0.9365
Epoch 6/30
40/40 [==============================] - 0s 10ms/step - loss: 0.2146 - accuracy: 0.9061 - val_loss: 0.2166 - val_accuracy: 0.9365
Epoch 7/30
40/40 [==============================] - 0s 9ms/step - loss: 0.2079 - accuracy: 0.9132 - val_loss: 0.2146 - val_accuracy: 0.9333
Epoch 8/30
40/40

### metrics=['f1_score']

In [106]:
model_1.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=[''])

In [108]:
# history = model_1.fit(x_train_dataset, y_train_dataset, batch_size=32, epochs=30, validation_data=(x_test_dataset, y_test_dataset))

# Расширение данных на основе нормативной документации

In [ ]:
df_extension = pd.read_excel('/content/drive/MyDrive/NU_files/стажировка/spr_mkb10.xls')

In [ ]:
df_extension.head()

In [ ]:
df_extension = df_extension.drop('class_id', axis=1)
df_extension.head()

In [ ]:
# list_1 = df_extension['mkb_cod']

In [ ]:
# def extension(arg):
#     mkb = df_extension['mkb_cod']
#     if mkb in list_1:
#         df_extension['factors_diseased'] 

In [ ]:
vocabulary_mkb[:5]

In [ ]:
print(vocabulary_mkb.index('A00'))
print(vocabulary_mkb.index('A09'))

In [ ]:
list_1 = vocabulary_mkb[0:68]
list_1

In [ ]:
import random

random.choice(list_1)

In [ ]:
vocabulary_mkb.index('E84')

In [ ]:
vocabulary_mkb[vocabulary_mkb.index('E84'):2420]
# print(vocabulary_mkb.index('A09'))